In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import mlflow
import dvc

from sklearn.preprocessing import (
    OneHotEncoder,
    LabelEncoder,
    MinMaxScaler,
    StandardScaler
)
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import (
    make_pipeline,
    Pipeline
)
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from sklearn.metrics import ConfusionMatrixDisplay
import mlflow 
from feature_engine.outliers.winsorizer import Winsorizer
import pandas as pd



In [ ]:
uri = "https://raw.githubusercontent.com/SandeepSuthar169/Datasets/main/bank.csv"
df = pd.read_csv(uri)

print(df.head())


df['marital'] = df['marital'].replace({
    'divorced': 1,
    'single': 2,
    'married': 3
})
df['marital'] = df['marital'].astype(int)

df['job'] = df['job'].replace({
    'unknown': 1,
    'unemployed': 1,
    'housemaid': 2,
    'student': 2,
    'self-employed': 3,
    'retired': 4,
    'services': 4,
    'admin.': 5,
    'technician': 5,
    'blue-collar': 5,
    'management': 6,
    'entrepreneur': 6   
    })
df['job'] = df['job'].astype(int)

df['poutcome'] = df['poutcome'].replace({
    'unknown': 1,
    'other': 1,
    'failure': 2,
    'success': 2
})
df['poutcome'] = df['poutcome'].astype(int)

df['education'] = df['education'].replace({
    'unknown': 1,
    'primary': 2,
    'tertiary': 3,
    'secondary': 4
})
df['education'] = df['education'].astype(int)


df['contact'] = df['contact'].replace({
    'unknown': 1,
    'cellular': 2,
    'telephone': 3   
})
df['contact'] = df['contact'].astype(int)



#deposit, month, loan, housing, default
prepro= ColumnTransformer(transformers=[
    ('one', OneHotEncoder(), ['default', 'housing', 'loan', 'month'])
], 
     remainder='passthrough'
)


pipe = Pipeline(steps =[
    ('prepro', prepro),
    ("classi", RandomForestClassifier(n_estimators=500 ))
])



train_data, test_data=train_test_split(df, random_state=42, test_size=0.2)




X_train = train_data.drop(columns=['deposit'])
y_train = train_data['deposit']

X_test = test_data.drop(columns=['deposit'])
y_test = test_data['deposit']



pipe.fit(X_train, y_train)



y_pred = pipe.predict(X_test)


acc = accuracy_score(y_test,y_pred)
#pre = precision_score(y_test, y_pred)
#recall = recall_score(y_test, y_pred)
#f1 = f1_score(y_test, y_pred)



print("acc:->",acc)
#print('pre:->', pre)
#print('recall:->', recall)
#print('f1 :->', f1)

   age         job  marital  education default  balance housing loan  contact  \
0   59      admin.  married  secondary      no     2343     yes   no  unknown   
1   56      admin.  married  secondary      no       45      no   no  unknown   
2   41  technician  married  secondary      no     1270     yes   no  unknown   
3   55    services  married  secondary      no     2476     yes   no  unknown   
4   54      admin.  married   tertiary      no      184      no   no  unknown   

   day month  duration  campaign  pdays  previous poutcome deposit  
0    5   may      1042         1     -1         0  unknown     yes  
1    5   may      1467         1     -1         0  unknown     yes  
2    5   may      1389         1     -1         0  unknown     yes  
3    5   may       579         1     -1         0  unknown     yes  
4    5   may       673         2     -1         0  unknown     yes  


C:\Users\Sande\AppData\Local\Temp\ipykernel_4796\1353751703.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['marital'] = df['marital'].replace({
C:\Users\Sande\AppData\Local\Temp\ipykernel_4796\1353751703.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['job'] = df['job'].replace({
C:\Users\Sande\AppData\Local\Temp\ipykernel_4796\1353751703.py:30: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(co

acc:-> 0.8423645320197044


In [4]:
import mlflow.sklearn

mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment('Grid_Serch_Values')

mlflow.sklearn.autolog()


params_grid = {
    "classi__n_estimators": list(range(300, 801, 100)),
    "classi__max_depth": [2, 3, 4, 5, 6, 7],
    "classi__min_samples_split": [2, 3, 4, 5] 
}


with mlflow.start_run(run_name="outer") as parent_run:
    grid_serch = GridSearchCV(estimator=pipe,
                              cv = 3,
                              param_grid=params_grid,
                              scoring='accuracy',
                              n_jobs=-1)
    
    grid_serch.fit(X_train, y_train)

with mlflow.start_run(run_name="best_model", nested=True): 
    mlflow.log_params(grid_serch.best_params_)
    mlflow.log_metric("best_accuracy", grid_serch.best_score_)
    mlflow.sklearn.log_model(sk_model=grid_serch.best_estimator_, artifact_path="best_model")

2025/05/16 09:27:47 INFO mlflow.tracking.fluent: Experiment with name 'Grid_Serch_Values' does not exist. Creating a new experiment.
2025/05/16 09:27:49 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Sande\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/05/16 09:29:18 WARNI

🏃 View run nimble-gnu-906 at: http://127.0.0.1:5000/#/experiments/424559000255678126/runs/7a6e1fe34b6f413089bfed3b09d2918d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/424559000255678126
🏃 View run aged-trout-94 at: http://127.0.0.1:5000/#/experiments/424559000255678126/runs/693397eeab454374a62e81bad67a41ff
🏃 View run wistful-croc-285 at: http://127.0.0.1:5000/#/experiments/424559000255678126/runs/817b2944243240c3b2457c988a87aab5
🏃 View run persistent-rat-67 at: http://127.0.0.1:5000/#/experiments/424559000255678126/runs/b6e2305677f44ffa96ca2102bee6dda9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/424559000255678126
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/424559000255678126
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/424559000255678126
🏃 View run capable-sponge-69 at: http://127.0.0.1:5000/#/experiments/424559000255678126/runs/be0905d9b8104121963b2756357bee66
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/424559000255

2025/05/16 09:29:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run best_model at: http://127.0.0.1:5000/#/experiments/424559000255678126/runs/5c0a2f3ec2a2499cafbdb62c77482537
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/424559000255678126
